Importing necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from transformers import BertTokenizerFast
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSequenceClassification, pipeline, AutoModelForSeq2SeqLM

Loading the pretrained Bert model for Named Entity Recognition (NER)

In [ ]:
# Load pre-trained BERT model for NER
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
# Create the NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load pre-trained BERT model for sentiment analysis

In [ ]:
sentiment_tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
sentiment_model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
# Create the sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model=sentiment_model, tokenizer=sentiment_tokenizer)

Function for web crawling

In [ ]:
articles = {}
def get_links(link):

  return_links = []
  return_news = []
  r = requests.get(link)
  soup = BeautifulSoup(r.content, "html.parser")
  # soup = BeautifulSoup(r.content, "lxml")
  heading=soup.find('title')
  print(heading.get_text())

  if r.status_code != 200:
    print("Error. Something is wrong here")
  else:
    #Scraping news from MyJoyOnline
    if link == 'https://www.myjoyonline.com/':
      start = soup.findAll('div', class_='home-post-list-title')
      for news in start:
        a_tag = news.find('a')
        if a_tag:
            link = a_tag['href']
            # Find the h4 tag within the a tag to get the news text
            news_text = a_tag.find('h4').text
             # Add the news text and link to the dictionary
            articles[news_text] = {"link": link}

            # Fetch the content of the individual article page
            article_response = requests.get(link)
            article_soup = BeautifulSoup(article_response.content, 'html.parser')

            # Find the summary in the individual article page
            summary = article_soup.findAll('div', id="article-text", class_="mt-3 article-text")
            article_summary = ""
            for s in summary:
                p_tags = s.find_all('p')
                for p in p_tags:
                    article_summary += p.text.strip() + " "

            # Add the summary to the existing dictionary entry
            articles[news_text] = {"link": link, "summary": article_summary.strip()}

      # Print articles with summaries
      for news_text, data in articles.items():
          if 'summary' in data:
            print(f"Title: {news_text}")
            print(f"Link: {data['link']}")
            print(f"News: {data['summary']}")
            print("----")

    #Scraping news from Pulse Ghana
    elif link == "https://www.pulse.com.gh/":
      for links in soup.findAll('a', attrs={'href': re.compile("^http")}):
        news_text = links.get('title')
        return_news.append(news_text)
        link = links.get('href')
        return_links.append(link)
        articles[news_text] = {"link": link}

        # Fetch the content of the individual article page
        article_response = requests.get(link)
        article_soup = BeautifulSoup(article_response.content, 'html.parser')

        # Find the summary in the individual article page
        summary_start = article_soup.findAll('article')
        summary = article_soup.findAll('div', id="lead", class_="article-perex")
        article_summary = ""
        for s in summary:
            p_tags = s.find_all('p')
            for p in p_tags:
                article_summary += p.text.strip() + " "

        # Only add articles with summaries
        if article_summary != "":
            articles[news_text] = {"link": link, "summary": article_summary.strip()}
        else:
          del articles[news_text]

      # Print articles with summaries
      for news_text, data in articles.items():
          if 'summary' in data:
              print(f"Title: {news_text}")
              print(f"Link: {data['link']}")
              print(f"Summary: {data['summary']}")
              print("----")


    #Scraping news from Ghanaian Times
    elif link == "https://ghanaiantimes.com.gh/":
          start = soup.findAll('div', class_="post-details")
          next= soup.findAll('h2', class_="post-title")
          for links in next:
            a_tag = links.find('a')
            if a_tag:
                link = a_tag['href']
                # Find the h4 tag within the a tag to get the news text
                news_text = a_tag.find('h2')

                 # Add the news text and link to the dictionary
                articles[news_text] = {"link": link}

                # Fetch the content of the individual article page
                article_response = requests.get(link)
                article_soup = BeautifulSoup(article_response.content, 'html.parser')

                # Find the summary in the individual article page
                summary_start = article_soup.findAll('article', id="the-post", class_="container-wrapper post-content tie-standard")
                summary = article_soup.findAll('div', class_="entry-content entry clearfix is-expanded")
                article_summary = ""
                for s in summary:
                    p_tags = s.find_all('p')
                    print(p_tags)
                    for p in p_tags:
                        article_summary += p.text.strip() + " "

                    # Add the summary to the existing dictionary entry
                    articles[news_text]["summary"] = article_summary.strip()

            # Print articles with summaries
            for news_text, data in articles.items():
                if 'summary' in data:
                    print(f"Title: {news_text}")
                    print(f"Link: {data['link']}")
                    print(f"Summary: {data['summary']}")
                    print("----")


    #Scraping from Yen news
    elif link == "https://yen.com.gh/ghana/":
      start = soup.findAll('article', class_='c-article-card')
      for links in start:
            a_tag = links.find('a')
            if a_tag:
                link = a_tag['href']
                # Find the h4 tag within the a tag to get the news text
                news_text = a_tag.find('span').get_text()
                # Add the news text and link to the dictionary
                articles[news_text] = {"link": link}

                # Fetch the content of the individual article page
                article_response = requests.get("https://yen.com.gh/ghana/")
                article_soup = BeautifulSoup(article_response.content, 'html.parser')

                # Find the summary in the individual article page (adjust the selector)
                summary_elements = article_soup.find_all('div', class_='post__content')
                article_summary = ""
                for s in summary_elements:
                    article_summary += s.text.strip() + " "
                articles[news_text]['summary'] = article_summary.strip()

            # Print articles with summaries
            for news_text, data in articles.items():
                if 'summary' in data:
                    print(f"Title: {news_text}")
                    print(f"Link: {data['link']}")
                    print(f"Summary: {data['summary']}")
                    print("----")
                else:
                    print(f"Title: {news_text}")
                    print(f"Link: {data['link']}")
                    print("----")


    #Scraping news from GBC Ghana
    elif link == "https://www.gbcghanaonline.com/":
        start = soup.findAll('div', class_='elementor-post__text')
        for links in start:
            a_tag = links.find('a')
            if a_tag:
                link = a_tag['href']
                # Find the h4 tag within the a tag to get the news text
                news_text = a_tag.text.strip()
                # Add the news text and link to the dictionary
                articles[news_text] = {"link": link}

                # Fetch the content of the individual article page
                article_response = requests.get(link)
                article_soup = BeautifulSoup(article_response.content, 'html.parser')

                # Find the summary in the individual article page
                sum_start = article_soup.findAll('div', class_='e-con-inner')
                summary = article_soup.findAll('div', class_='elementor-widget-container')
                article_summary = ""
                for s in summary:
                    p_tags = s.find_all('p')
                    for p in p_tags:
                        article_summary += p.text.strip() + " "
                # Add the summary to the existing dictionary entry
                articles[news_text] = {"link": link, "summary": article_summary.strip()}

    # Print articles with summaries
    for news_text, data in articles.items():
        if 'summary' in data:
            print(f"Title: {news_text}")
            print(f"Link: {data['link']}")
            print(f"Summary: {data['summary']}")
            print("----")

  return articles

Loading the pretrained Bart model for text summarization

In [ ]:
model_name = 'facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(model_name)
summarizer = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Function for text summarization

In [ ]:
summarizer = pipeline('summarization', model=model_name, tokenizer= tokenizer)

def summarize_articles(articles):
    summarized_articles = {}
    for news_text, data in articles.items():
        if 'summary' in data:
            try:
                summarized_text = summarizer(data['summary'], max_length=125, min_length=25, do_sample=False)
                if summarized_text and len(summarized_text) > 0:
                    summarized_articles[news_text] = {
                        'link': data['link'],
                        'news': data['summary'],
                        'summary': summarized_text[0]['summary_text']
                    }
                else:
                    print(f"Summarizer returned an empty result for article: {news_text}")
            except Exception as e:
                print(f"Error summarizing article {news_text}: {str(e)}")
                summarized_articles[news_text] = {
                    'link': data['link'],
                    'news': data['summary'],
                    'summary': "Summary generation failed."
                }
    return summarized_articles

Applying the function to get news from various websites as well as summarizing the news

In [ ]:
link = "https://www.pulse.com.gh/"
articles = get_links(link)
summarized_articles = summarize_articles(articles)

Breaking News & Top Stories | Pulse Ghana


Your max_length is set to 125, but your input_length is only 39. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


Title: Prof. Oquaye credits NPP for human rights in Ghana, criticises Nkrumah's regime
Link: https://www.pulse.com.gh/news/politics/prof-oquaye-credits-npp-for-human-rights-in-ghana-criticises-nkrumahs-regime/vndsz7d
Summary: Former Speaker of Parliament, Prof. Mike Oquaye, has stated that Ghanaians owe the human rights they currently enjoy to the efforts of the New Patriotic Party (NPP).
----
Title: 7 things to never say to someone going through depression
Link: https://www.pulse.ng/lifestyle/7-things-to-never-say-to-someone-going-through-depression/zynennd
Summary: This is a complex mental health condition that affects millions of people worldwide.
----
Title: Judicial Service set to establish debt recovery courts – Chief Justice
Link: https://www.pulse.com.gh/news/local/judicial-service-set-to-establish-debt-recovery-courts-chief-justice/plet6sr
Summary: The Judicial Service is set to establish small claim debt recovery courts to recover monies borrowed by customers from financial i

Your max_length is set to 125, but your input_length is only 16. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)
Your max_length is set to 125, but your input_length is only 39. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)
Your max_length is set to 125, but your input_length is only 56. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)
Your max_length is set to 125, but your input_length is only 45. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=22)
Your 

In [ ]:
# Print summarized articles to verify
print(f"Number of summarized articles: {len(summarized_articles)}")
for title, content in summarized_articles.items():
    print(f"Title: {title}, Link: {content['link']}, Summary: {content.get('summary', 'No summary')}")

Number of summarized articles: 28
Title: Prof. Oquaye credits NPP for human rights in Ghana, criticises Nkrumah's regime, Link: https://www.pulse.com.gh/news/politics/prof-oquaye-credits-npp-for-human-rights-in-ghana-criticises-nkrumahs-regime/vndsz7d, Summary: Prof. Mike Oquaye says Ghanaians owe the human rights they currently enjoy to the efforts of the New Patriotic Party (NPP)
Title: 7 things to never say to someone going through depression, Link: https://www.pulse.ng/lifestyle/7-things-to-never-say-to-someone-going-through-depression/zynennd, Summary: This is a complex mental health condition that affects millions of people worldwide. It is a condition that can lead to depression, anxiety and other mental health problems.
Title: Judicial Service set to establish debt recovery courts – Chief Justice, Link: https://www.pulse.com.gh/news/local/judicial-service-set-to-establish-debt-recovery-courts-chief-justice/plet6sr, Summary: The Judicial Service is set to establish small claim d

In [ ]:
link = "https://www.myjoyonline.com/"
articles = get_links(link)
summarized_articles = summarize_articles(articles)

MyJoyOnline - Ghana News | Ghana's most comprehensive website. Independent, Fearless and Credible journalism


Your max_length is set to 125, but your input_length is only 39. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


Title: Prof. Oquaye credits NPP for human rights in Ghana, criticises Nkrumah's regime
Link: https://www.pulse.com.gh/news/politics/prof-oquaye-credits-npp-for-human-rights-in-ghana-criticises-nkrumahs-regime/vndsz7d
News: Former Speaker of Parliament, Prof. Mike Oquaye, has stated that Ghanaians owe the human rights they currently enjoy to the efforts of the New Patriotic Party (NPP).
----
Title: 7 things to never say to someone going through depression
Link: https://www.pulse.ng/lifestyle/7-things-to-never-say-to-someone-going-through-depression/zynennd
News: This is a complex mental health condition that affects millions of people worldwide.
----
Title: Judicial Service set to establish debt recovery courts – Chief Justice
Link: https://www.pulse.com.gh/news/local/judicial-service-set-to-establish-debt-recovery-courts-chief-justice/plet6sr
News: The Judicial Service is set to establish small claim debt recovery courts to recover monies borrowed by customers from financial institutio

Your max_length is set to 125, but your input_length is only 16. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)
Your max_length is set to 125, but your input_length is only 39. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)
Your max_length is set to 125, but your input_length is only 56. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)
Your max_length is set to 125, but your input_length is only 45. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=22)
Your 

In [ ]:
print(f"Number of summarized articles: {len(summarized_articles)}")
for title, content in summarized_articles.items():
    print(f"Title: {title}, Link: {content['link']}, Summary: {content.get('summary', 'No summary')}")

Number of summarized articles: 39
Title: Prof. Oquaye credits NPP for human rights in Ghana, criticises Nkrumah's regime, Link: https://www.pulse.com.gh/news/politics/prof-oquaye-credits-npp-for-human-rights-in-ghana-criticises-nkrumahs-regime/vndsz7d, Summary: Prof. Mike Oquaye says Ghanaians owe the human rights they currently enjoy to the efforts of the New Patriotic Party (NPP)
Title: 7 things to never say to someone going through depression, Link: https://www.pulse.ng/lifestyle/7-things-to-never-say-to-someone-going-through-depression/zynennd, Summary: This is a complex mental health condition that affects millions of people worldwide. It is a condition that can lead to depression, anxiety and other mental health problems.
Title: Judicial Service set to establish debt recovery courts – Chief Justice, Link: https://www.pulse.com.gh/news/local/judicial-service-set-to-establish-debt-recovery-courts-chief-justice/plet6sr, Summary: The Judicial Service is set to establish small claim d

In [ ]:
link = "https://yen.com.gh/ghana/"
articles = get_links(link)
summarized_articles = summarize_articles(articles)

Ghana News - Latest News and Reports. Daily updates on Yen.com.gh
Title: Nana Akua Addo inspires Mam Dollars' 50th birthday look, peeps gush over photos
Link: https://yen.com.gh/entertainment/celebrities/263566-mama-dollars-copies-nana-akua-addos-2019-style-50th-birthday-peeps-drool-photos/
Summary: 
----
Title: Nana Akua Addo inspires Mam Dollars' 50th birthday look, peeps gush over photos
Link: https://yen.com.gh/entertainment/celebrities/263566-mama-dollars-copies-nana-akua-addos-2019-style-50th-birthday-peeps-drool-photos/
Summary: 
----
Title: Shatta Wale calls out Stonebwoy and Sarkodie, sends a message to Ghanaian promoters
Link: https://yen.com.gh/entertainment/263573-shatta-wale-calls-stonebwoy-sarkodie-sends-a-message-ghanaian-promoters/
Summary: 
----
Title: Nana Akua Addo inspires Mam Dollars' 50th birthday look, peeps gush over photos
Link: https://yen.com.gh/entertainment/celebrities/263566-mama-dollars-copies-nana-akua-addos-2019-style-50th-birthday-peeps-drool-photos/
S

Your max_length is set to 125, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


Title: Nana Akua Addo inspires Mam Dollars' 50th birthday look, peeps gush over photos
Link: https://yen.com.gh/entertainment/celebrities/263566-mama-dollars-copies-nana-akua-addos-2019-style-50th-birthday-peeps-drool-photos/
Summary: 
----
Title: Shatta Wale calls out Stonebwoy and Sarkodie, sends a message to Ghanaian promoters
Link: https://yen.com.gh/entertainment/263573-shatta-wale-calls-stonebwoy-sarkodie-sends-a-message-ghanaian-promoters/
Summary: 
----
Title: Ranking the Top 12 Footballers With Most Free Kick Goals in History
Link: https://yen.com.gh/sports/football/263577-ranking-top-12-footballers-free-kick-goals-history/
Summary: 
----
Title: 2019 Di Asa winner laments being single, appeals to Ghanaian men to date her: "I am a full package"
Link: https://yen.com.gh/people/263545-2019-di-asa-laments-single-appeals-guys-date-video/
Summary: 
----
Title: Parliament to pass LI that would allow MPs and ministers to use sirens, ignore speed limits
Link: https://yen.com.gh/ghana/2

Your max_length is set to 125, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 125, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 125, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 125, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_len

In [ ]:
link = "https://www.gbcghanaonline.com/"
articles = get_links(link)
summarized_articles = summarize_articles(articles)
summarized_articles

GBC Ghana Online - The Nation's Broadcaster | Breaking News from Ghana, Business, Sports, Entertainment, Fashion and Video News


Your max_length is set to 125, but your input_length is only 39. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


Title: Prof. Oquaye credits NPP for human rights in Ghana, criticises Nkrumah's regime
Link: https://www.pulse.com.gh/news/politics/prof-oquaye-credits-npp-for-human-rights-in-ghana-criticises-nkrumahs-regime/vndsz7d
Summary: Former Speaker of Parliament, Prof. Mike Oquaye, has stated that Ghanaians owe the human rights they currently enjoy to the efforts of the New Patriotic Party (NPP).
----
Title: 7 things to never say to someone going through depression
Link: https://www.pulse.ng/lifestyle/7-things-to-never-say-to-someone-going-through-depression/zynennd
Summary: This is a complex mental health condition that affects millions of people worldwide.
----
Title: Judicial Service set to establish debt recovery courts – Chief Justice
Link: https://www.pulse.com.gh/news/local/judicial-service-set-to-establish-debt-recovery-courts-chief-justice/plet6sr
Summary: The Judicial Service is set to establish small claim debt recovery courts to recover monies borrowed by customers from financial i

Your max_length is set to 125, but your input_length is only 16. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)
Your max_length is set to 125, but your input_length is only 39. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)
Your max_length is set to 125, but your input_length is only 56. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)
Your max_length is set to 125, but your input_length is only 45. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=22)
Your 

Error summarizing article Book Review: Revolution and Democracy in Ghana: The Politics of Jerry Rawlings: index out of range in self


{"Prof. Oquaye credits NPP for human rights in Ghana, criticises Nkrumah's regime": {'link': 'https://www.pulse.com.gh/news/politics/prof-oquaye-credits-npp-for-human-rights-in-ghana-criticises-nkrumahs-regime/vndsz7d',
  'news': 'Former Speaker of Parliament, Prof. Mike Oquaye, has stated that Ghanaians owe the human rights they currently enjoy to the efforts of the New Patriotic Party (NPP).',
  'summary': 'Prof. Mike Oquaye says Ghanaians owe the human rights they currently enjoy to the efforts of the New Patriotic Party (NPP)'},
 '7 things to never say to someone going through depression': {'link': 'https://www.pulse.ng/lifestyle/7-things-to-never-say-to-someone-going-through-depression/zynennd',
  'news': 'This is a complex mental health condition that affects millions of people worldwide.',
  'summary': 'This is a complex mental health condition that affects millions of people worldwide. It is a condition that can lead to depression, anxiety and other mental health problems.'},
 

Storing all news articles in a dataframe

In [ ]:
data = {
    "Title": list(articles.keys()),
    "Link": [articles[title]['link'] for title in articles.keys()],
    "Summary": [summarized_articles.get(title, {}).get('summary','') for title in articles.keys()]
}

In [ ]:
articles = pd.DataFrame(data)
articles

,Title,Link,Summary
0,Prof. Oquaye credits NPP for human rights in G...,https://www.pulse.com.gh/news/politics/prof-oq...,Prof. Mike Oquaye says Ghanaians owe the human...
1,7 things to never say to someone going through...,https://www.pulse.ng/lifestyle/7-things-to-nev...,This is a complex mental health condition that...
2,Judicial Service set to establish debt recover...,https://www.pulse.com.gh/news/local/judicial-s...,The Judicial Service is set to establish small...
3,Ken Agyapong urges NPP to admit mistakes and a...,https://www.pulse.com.gh/news/politics/ken-agy...,"Member of Parliament for Assin Central, Kenned..."
4,Retract your statement or face my wrath - Adom...,https://www.pulse.com.gh/news/filla/retract-yo...,Prophet Stephen Adom Kyei-Duah has issued a s...
...,...,...,...
66,Book Review: Revolution and Democracy in Ghana...,https://www.gbcghanaonline.com/features/book-j...,Summary generation failed.
67,Founders’ Day: President Akufo-Addo calls for ...,https://www.gbcghanaonline.com/general/founder...,President Akufo-Addo has encouraged Ghanaians ...
68,Bono NDC Youth Wing intensifies electioneering...,https://www.gbcghanaonline.com/general-news/nd...,The Bono Regional Youth Wing of the National D...
69,Mop-up Registration: Kadjebi registers 68 vote...,https://www.gbcghanaonline.com/news/politics/v...,The Kadjebi District Office of the Electoral C...


Vectorizing the words in each article summary

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorization
vectorizer = TfidfVectorizer(max_df=0.90, min_df=1, stop_words='english')
X = vectorizer.fit_transform(articles['Summary'])

# Check the shape of the matrix
print(X.shape)


(71, 863)


Clustering the vectorized articles

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np

# DBSCAN clustering
db = DBSCAN(eps=0.7, min_samples=3, metric='cosine').fit(X)

# Add cluster labels to the DataFrame
articles['Cluster'] = db.labels_

# Check the distribution of clusters
print(articles['Cluster'].value_counts())


Cluster
-1    56
 1    12
 0     3
Name: count, dtype: int64


Using cosine similarity to find which articles are relevant(THE HOTTEST! TOPIC IN TOWN)

In [ ]:
# print(trending_articles_df.to_string())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Separate articles
yen_articles = articles[articles['Link'].str.contains('yen.com.gh')]
other_articles = articles[~articles['Link'].str.contains('yen.com.gh')]

# Fit vectorizer on all data to ensure consistent feature space
vectorizer = TfidfVectorizer(max_df=0.90, min_df=1, stop_words='english')
vectorizer.fit(articles['Title'].tolist() + articles['Summary'].tolist()) # Fit on both titles and summaries

# Transform titles and summaries separately using the fitted vectorizer
yen_vectors = vectorizer.transform(yen_articles['Title'])
other_vectors = vectorizer.transform(other_articles['Summary'])

# Combine vectors (adjust as needed)
all_vectors = np.concatenate((yen_vectors.toarray(), other_vectors.toarray()), axis=0)

# Check the shape of the matrix
print(all_vectors.shape)

(71, 1114)


In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np

# DBSCAN clustering
dbs = DBSCAN(eps=0.7, min_samples=3, metric='cosine').fit(all_vectors)

# Add cluster labels to the DataFrame
articles['Cluster'] = db.labels_

# Check the distribution of clusters
print(articles['Cluster'].value_counts())


Cluster
-1    56
 1    12
 0     3
Name: count, dtype: int64


In [ ]:
def analyze_sentiment_and_cluster(articles_df, sentiment_pipeline, max_length=512): # Added max_length parameter
    # Perform sentiment analysis on all articles
    articles_df['Sentiment'] = articles_df['Summary'].apply(lambda x: sentiment_pipeline(x[:max_length])[0]['label'])

    # Filter articles that are in clusters other than -1
    filtered_df = articles_df[articles_df['Cluster'] != -1]


    for cluster_id in filtered_df['Cluster'].unique():
        cluster_articles = filtered_df[filtered_df['Cluster'] == cluster_id]
        print(f"Cluster ID: {cluster_id}")
        for idx, row in cluster_articles.iterrows():
            print(f"Title: {row['Title']}")
            print(f"Link: {row['Link']}")
            print(f"Summary: {row['Summary']}")
            print(f"Sentiment: {row['Sentiment']}")
            print("----")

In [ ]:
# Perform sentiment analysis on articles
analyze_sentiment_and_cluster(articles, sentiment_pipeline)

Cluster ID: 0
Title: Why share in Nkrumah’s achievements after opposing his idea of independence? – Selorm Branttie
Link: https://www.myjoyonline.com/why-share-in-nkrumahs-achievements-after-opposing-his-idea-of-independence-selorm-branttie/
Summary: Selorm Branttie, Vice President of IMANI Africa, has condemned attempts by some persons to alter Ghana's historical records. His remark is in response to President Akufo-Addo’s recent statement rejecting the notion that Ghana was solely founded by Kwame Nkrumah.
Sentiment: 1 star
----
Title: Others contributed to liberation from colonialism, Nkrumah’s vision shaped modern Ghana – Kwame Sarpong
Link: https://www.myjoyonline.com/others-contributed-to-liberation-from-colonialism-nkrumahs-vision-shaped-modern-ghana-kwame-sarpong/
Summary: A research fellow at CDD-Ghana, Kwame Sarpong Asiedu, has stated that the Ghana we know today is the result of Kwame Nkrumah's vision. This statement follows President Akufo-Addo’s recent remarks rejecting th

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from transformers import pipeline

# Function to create a prediction with dropout
def predict_with_dropout(pipeline, text, dropout=0.5, num_iter=20):
    # Set the model in evaluation mode
    model = pipeline.model
    model.train()  # Enable dropout

    # List to store predictions
    predictions = []

    for _ in range(num_iter):
        with torch.no_grad():
            inputs = pipeline.tokenizer(text, return_tensors='pt', truncation=True, padding=True)
            outputs = model(**inputs)
            probs = F.softmax(outputs.logits, dim=-1).cpu().numpy()
            predictions.append(probs)

    return np.array(predictions)

# Example usage
texts = articles['Summary'].tolist()
dropout = 0.5
num_iter = 20

# Get predictions with dropout
predictions = np.array([predict_with_dropout(sentiment_pipeline, text, dropout, num_iter) for text in texts])

# Calculate the confidence intervals
ci = 0.8
lower_lim = np.quantile(predictions, 0.5-ci/2, axis=2)
upper_lim = np.quantile(predictions, 0.5+ci/2, axis=2)

# Print the confidence intervals for each text
for i, text in enumerate(texts):
    mean_prediction = predictions[i].mean(axis=0)
    lower_bound = lower_lim[i]
    upper_bound = upper_lim[i]

    print(f"Text: {text}")
    print(f"Prediction: {mean_prediction}")
    print(f"80% Confidence Interval: ({lower_bound}, {upper_bound})")
    print("----")

Text: Prof. Mike Oquaye says Ghanaians owe the human rights they currently enjoy to the efforts of the New Patriotic Party (NPP)
Prediction: [[0.30637276 0.30084866 0.3927786 ]]
80% Confidence Interval: ([[0.30984583 0.31037381 0.37978038]
 [0.31064275 0.27401391 0.41534334]
 [0.30826929 0.2964645  0.39526621]
 [0.31692153 0.31299889 0.37007961]
 [0.32116035 0.28909025 0.38974941]
 [0.2992444  0.30707344 0.39368212]
 [0.32064757 0.29754189 0.38181058]
 [0.30876708 0.31793249 0.37330043]
 [0.29356918 0.30588958 0.40054128]
 [0.29771021 0.3034969  0.39879286]
 [0.31193849 0.29174903 0.39631248]
 [0.2949442  0.28303728 0.42201856]
 [0.30291665 0.29844949 0.39863387]
 [0.31558174 0.30241069 0.38200757]
 [0.31241468 0.30951256 0.3780728 ]
 [0.31929907 0.27071601 0.40998498]
 [0.29059672 0.32998961 0.37941372]
 [0.28665632 0.30528527 0.40805843]
 [0.29743108 0.31648836 0.38608056]
 [0.30889827 0.29445913 0.39664257]], [[0.30984583 0.31037381 0.37978038]
 [0.31064275 0.27401391 0.41534334]
 [

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

num_classes = 10

# Fit and transform the data
train_data = vectorizer.fit_transform(articles).toarray()
train_data.shape

# Convert labels to NumPy array
train_labels = np.array(articles['Cluster'])

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=()),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2, callbacks=[early_stopping])

SyntaxError: invalid syntax (<ipython-input-80-0e32307b0edd>, line 13)